This notebook was run in Julia 1.6. The following Julia packages are used in this note book.

DifferentialEquations, Flux, DiffEqFlux, Optim, DiffEqSensitivity, Plots, OrdinaryDiffEq, Zygote, StaticArrays, LinearAlgebra, BenchmarkTools, PaddedViews

These packages can be installed using the following commands:
```
using Pkg;
Pkg.add(["OrdinaryDiffEq","DifferentialEquations", "Flux", "DiffEqFlux", "Optim", "DiffEqSensitivity", "Plots", "Zygote", "StaticArrays", "BenchmarkTools", "PaddedViews","NLopt","BlackBoxOptim", "DiffEqBase"]);
```

In [ ]:
gr(show = :ijulia)

In [ ]:
using DifferentialEquations, Flux, DiffEqFlux, DiffEqSensitivity, Plots, OrdinaryDiffEq, Optim, Zygote, StaticArrays, LinearAlgebra, BenchmarkTools, PaddedViews, PlotThemes
using NLopt,BlackBoxOptim, DiffEqBase, SparseArrays

In [ ]:
Defining the constants needed for all leakage optimizations.
const σ0 = Hermitian(Complex{Float64}[1 0; 0 1]);
const σx = Hermitian(Complex{Float64}[0 1; 1 0]);
const σy = Hermitian(Complex{Float64}[0 -im; im 0]);
const σz = Hermitian(Complex{Float64}[1 0; 0 -1]);
const III = kron(σ0,kron(σ0,σ0));
const couple1=Diagonal([0,8,16]);
const couple2=Diagonal([2,14,26]);
const couple3=Diagonal([4,20,36]);
const couple4=Diagonal([6,26,46]);
const couple5=Diagonal([8,32,56]);
const N=3;
const a = (sparse([i for i=1:N-1],[i for i=2:N],[√i+0im for i=1:N-1], N, N));
const at = (sparse([i for i=2:N],[i for i=1:N-1],[√i+0im for i=1:N-1], N, N));
const a0 = Diagonal(sparse([i for i=1:N],[i for i=1:N],[1.0+0im for i=1:N], N, N));
const n = Diagonal(at*a);
const η = Diagonal(at*at*a*a); # non linearity of transmons also equal to at*a*(at*a-id)
const id = Matrix{Complex{Float64}}(I, N, N); # identity
const Δ=10;
const Ωmax = 4.0;
const Jmax = 1.0;
const Utarget = σ0#exp(im*π/2*σz);
int(x) = floor(Int, x);

In the next section the blocks define the constants needed for each different vertex type (2,3, or 4 nearest neighbors).
First is 2 neighborgrs

In [ ]:
# constants for 2 nearest neighbors
const III = kron(σ0,kron(σ0,σ0));
const III0 = kron(σ00,kron(σ00,σ00));
const IIX = kron(σ0,kron(σ0,σx1));
const IIY = kron(σ0,kron(σ0,σy1));
const IIX2 = kron(σ0,kron(σ0,σx2));
const IIY2 = kron(σ0,kron(σ0,σy2));
const IZZ = kron(σ0,kron(σz1,σz1));
const ZIZ = kron(σz1,kron(σ0,σz1));
const LII = kron(σp2,kron(σ0,σ0));
const ILI = kron(σ0,kron(σp2,σ0));
const IIL = kron(σ0,kron(σ0,σp2));
const u0 = Complex{Float64}[σ0; σ0; σ0; 0 0 0];
const T=3*π;
const numberF=100;
tspan = (0.0, T);
const U0 = III;
const UtargetAll = exp(im*pi/4*IIX);
const Bigcouple=Diagonal([0, 4, 8, 2, 10, 18, 4, 16, 28, 2, 10, 18, 4, 16, 28, 6, 22, 38, 4, 16, 28, 6, 22, 38, 8, 28, 48]);
pinitial=ones(numberF)./numberF.*7;

In [ ]:
# constants for 3 nearest neighbors
const IIII = kron(σ0,kron(σ0,kron(σ0,σ0)));
const IIIX = kron(σ0,kron(σ0,kron(σ0,σx1)));
const IIIY = kron(σ0,kron(σ0,kron(σ0,σy1)));
const IIIX2 = kron(σ0,kron(σ0,kron(σ0,σx2)));
const IIIY2 = kron(σ0,kron(σ0,kron(σ0,σy2)));
const ZIIZ = kron(σz1,kron(σ0,kron(σ0,σz1)));
const IIZZ = kron(σ0,kron(σ0,kron(σz1,σz1)));
const IZIZ = kron(σ0,kron(σz1,kron(σ0,σz1)));
const IIIL=kron(σ0,kron(σ0,kron(σ0,σp2)));
const IILI=kron(σ0,kron(σ0,kron(σp2,σ0)));
const ILII=kron(σ0,kron(σp2,kron(σ0,σ0)));
const LIII=kron(σp2,kron(σ0,kron(σ0,σ0)));
const IIII0 = kron(σ00,kron(σ00,kron(σ00,σ00)));
const u0 = Complex{Float64}[σ0; σ0; σ0; σ0; 0 0 0];
const T=5*π;
const numberF=200;
tspan = (0.0, T);
const U0 = IIII;
const UtargetAll = exp(im*pi/4*IIIX);
const Bigcouple=Diagonal([0, 6, 12, 2, 12, 22, 4, 18, 32, 2, 12, 22, 4, 18, 32, 6, 24, 42, 4, 18, 32, 6, 24, 42, 8, 30, 52, 2, 12, 22, 4, 18, 32, 6, 24, 42, 4, 18, 32, 6, 24, 42, 8, 30, 52, 6, 24, 42, 8, 30, 52, 10, 36, 62, 4, 18, 32, 6, 24, 42, 8, 30, 52, 6, 24, 42, 8, 30, 52, 10, 36, 62, 8, 30, 52, 10, 36, 62, 12, 42, 72]);
pinitial=ones(numberF)./numberF.*7;

In [ ]:
# constants for 4 nearest neighbors
const IIIII = kron(σ0,kron(σ0,kron(σ0,kron(σ0,σ0))));
const IIIIX = kron(σ0,kron(σ0,kron(σ0,kron(σ0,σx1))));
const IIIIY = kron(σ0,kron(σ0,kron(σ0,kron(σ0,σy1))));
const IIIIX2 = kron(σ0,kron(σ0,kron(σ0,kron(σ0,σx2))));
const IIIIY2 = kron(σ0,kron(σ0,kron(σ0,kron(σ0,σy2))));
const ZIIIZ = kron(σz1,kron(σ0,kron(σ0,kron(σ0,σz1))));
const ZIIZI = kron(σz1,kron(σ0,kron(σ0,kron(σz1,σ0))));
const IIZZI = kron(σ0,kron(σ0,kron(σz1,kron(σz1,σ0))));
const IZIZI = kron(σ0,kron(σz1,kron(σ0,kron(,σ0σz1))));
const IIILI=kron(σ0,kron(σ0,kron(σ0,kron(σp2,σ0))));
const IIIIL=kron(σ0,kron(σ0,kron(σ0,kron(σ0,σp2))));
const IILII=kron(σ0,kron(σ0,kron(σp2,kron(σ0,σ0))));
const ILIII=kron(σ0,kron(σp2,kron(σ0,kron(σ0,σ0))));
const LIIII=kron(σp2,kron(σ0,kron(σ0,kron(σ0,σ0))));
const IIIII0 = kron(σ00,kron(σ00,kron(σ00,kron(σ00,σ0))));
const u0 = Complex{Float64}[σ0; σ0; σ0; σ0; σ0; 0 0 0];
const T=7*π;
const numberF=200;
tspan = (0.0, T);
const U0 = IIIII;
const UtargetAll = exp(im*pi/4*IIIIX);
const Bigcouple=Diagonal([0, 6, 12, 2, 12, 22, 4, 18, 32, 2, 12, 22, 4, 18, 32, 6, 24, 42, 4, 18, 32, 6, 24, 42, 8, 30, 52, 2, 12, 22, 4, 18, 32, 6, 24, 42, 4, 18, 32, 6, 24, 42, 8, 30, 52, 6, 24, 42, 8, 30, 52, 10, 36, 62, 4, 18, 32, 6, 24, 42, 8, 30, 52, 6, 24, 42, 8, 30, 52, 10, 36, 62, 8, 30, 52, 10, 36, 62, 12, 42, 72]);
pinitial=ones(numberF)./numberF.*7;

In [ ]:
#Need to run this last since numberF needs to be defined first


Schrodinger equation \
For each number of nearest neighbors there is a different Schrodinger equation.\
First is the 2 vertices, then 3 vertices and finally 4 vertices as neighbor version.


In [ ]:
# Schrodinger equation for 2 nearest neighbors
# all these schrodinger equations calculate both the evolution operator as well as the integral of how much extra power over max power is used.
function schrodinger_f(u, p, t)
    @views @inbounds U1 = u[1:3, 1:3];# each seperate evolution operator in the commuting su(2) spaces
    @views @inbounds U2 = u[4:6, 1:3];
    @views @inbounds U3 = u[7:9, 1:3];
    Ωx, Ωy =[0 0]
    for i in 1:int(numberF/2)# construction of the instentanious value of Ωx and Ωy
        local sina=sin(pi*i*t/T)
        Ωx=Ωx+p[2*i-1]*sina
        Ωy=Ωy+p[2*i]*sina
    end 
    local H1 = Hermitian(Ωy/2*(σy1+σy2)+Ωx/2*(σx1+σx2)+Jmax/4*couple1+Δ*σp2);
    local H2 = Hermitian(Ωy/2*(σy1+σy2)+Ωx/2*(σx1+σx2)+Jmax/4*couple2+Δ*σp2);
    local H3 = Hermitian(Ωy/2*(σy1+σy2)+Ωx/2*(σx1+σx2)+Jmax/4*couple3+Δ*σp2);
    local power=relu(abs((Ωx^2+Ωy^2))-Ωmax^2) 
    return [-(im*H1)*U1;-(im*H2)*U2;-(im*H3)*U3; [power 0 0]]
end

ode_f = ODEProblem(schrodinger_f, u0, tspan, pinitial);

# loss function is a weighted sum of the trace fidelity and integrated amplitude over max
function cost(p)
    local ode_sol = solve(ode_f, BS5(), p=Complex{Float64}.(p), abstol=1.0e-7, reltol=1.0e-7) 
    local usol = last(ode_sol)
    @views @inbounds local Ugate1 = usol[1:2, 1:2]; 
    @views @inbounds local Ugate2 = usol[4:5, 1:2];  
    @views @inbounds local Ugate3 = usol[7:8, 1:2];  
    @views @inbounds local intPower=usol[10,1]
    @inbounds local loss = 0.003*abs(intPower)+abs(1.0-((tr(Ugate1*Utarget')/2)))+abs(1.0-((tr(Ugate2*Utarget')/2)))+abs(1.0-((tr(Ugate3*Utarget')/2)))
    return loss
end

plot_callback = function (p,l) #input current params p and cost l; stores and plots history of cost function throughout optimization
    append!(cost_history,l)
    plot(log10.(cost_history), xtickfont=18, ytickfont=18, legendfont=18);
    return false
end

In [ ]:
# Schrodinger equation for 3 nearest neighbors
function schrodinger_f(u, p, t)
    @views @inbounds U1 = u[1:3, 1:3];
    @views @inbounds U2 = u[4:6, 1:3];
    @views @inbounds U3 = u[7:9, 1:3];
    @views @inbounds U4 = u[10:12, 1:3];
    Ωx, Ωy =[0 0]
    for i in 1:int(numberF/2)
        local sina=sin(pi*i*t/T)
        Ωx=Ωx+p[2*i-1]*sina
        Ωy=Ωy+p[2*i]*sina
    end 
    local H1 = Hermitian(Ωy/2*(σy1+σy2)+Ωx/2*(σx1+σx2)+Jmax/4*couple1+Δ*σp2);
    local H2 = Hermitian(Ωy/2*(σy1+σy2)+Ωx/2*(σx1+σx2)+Jmax/4*couple2+Δ*σp2);
    local H3 = Hermitian(Ωy/2*(σy1+σy2)+Ωx/2*(σx1+σx2)+Jmax/4*couple3+Δ*σp2);
    local H4 = Hermitian(Ωy/2*(σy1+σy2)+Ωx/2*(σx1+σx2)+Jmax/4*couple4+Δ*σp2);
    local power=relu(abs((Ωx^2+Ωy^2))-Ωmax^2) 
    return [-(im*H1)*U1;-(im*H2)*U2;-(im*H3)*U3;-(im*H4)*U4; [power 0 0]]
end

ode_f = ODEProblem(schrodinger_f, u0, tspan, pinitial);

function cost(p)
    local ode_sol = solve(ode_f, BS5(), p=Complex{Float64}.(p), abstol=1.0e-8, reltol=1.0e-8) 
    local usol = last(ode_sol)
    @views @inbounds local Ugate1 = usol[1:2, 1:2]; 
    @views @inbounds local Ugate2 = usol[4:5, 1:2];  
    @views @inbounds local Ugate3 = usol[7:8, 1:2];
    @views @inbounds local Ugate4 = usol[10:11, 1:2];
    @views @inbounds local intPower=usol[13,1]
    @inbounds local loss = 0.003*abs(intPower)+(abs(1.0-((tr(Ugate1*Utarget')/2)))+abs(1.0-((tr(Ugate2*Utarget')/2)))+abs(1.0-((tr(Ugate3*Utarget')/2)))+abs(1.0-((tr(Ugate4*Utarget')/2)))) 
    return loss
end

plot_callback = function (p,l) #input current params p and cost l; stores and plots history of cost function throughout optimization
    append!(cost_history,l)
    plot(log10.(cost_history), xtickfont=18, ytickfont=18, legendfont=18);
    return false
end

In [ ]:
# Schrodinger equation for 4 nearest neighbors
function schrodinger_f(u, p, t)
    @views @inbounds U1 = u[1:3, 1:3];
    @views @inbounds U2 = u[4:6, 1:3];
    @views @inbounds U3 = u[7:9, 1:3];
    @views @inbounds U4 = u[10:12, 1:3];
    Ωx, Ωy =[0 0]
    for i in 1:int(numberF/2)
        local sina=sin(pi*i*t/T)
        Ωx=Ωx+p[2*i-1]*sina
        Ωy=Ωy+p[2*i]*sina
    end 
    local H1 = Hermitian(Ωy/2*(σy1+σy2)+Ωx/2*(σx1+σx2)+Jmax/4*couple1+Δ*σp2);
    local H2 = Hermitian(Ωy/2*(σy1+σy2)+Ωx/2*(σx1+σx2)+Jmax/4*couple2+Δ*σp2);
    local H3 = Hermitian(Ωy/2*(σy1+σy2)+Ωx/2*(σx1+σx2)+Jmax/4*couple3+Δ*σp2);
    local H4 = Hermitian(Ωy/2*(σy1+σy2)+Ωx/2*(σx1+σx2)+Jmax/4*couple4+Δ*σp2);
    local H5 = Hermitian(Ωy/2*(σy1+σy2)+Ωx/2*(σx1+σx2)+Jmax/4*couple5+Δ*σp2);
    local power=relu(abs((Ωx^2+Ωy^2))-Ωmax^2) 
   return [-(im*H1)*U1;-(im*H2)*U2;-(im*H3)*U3;-(im*H4)*U4;-(im*H5)*U5; [power 0 0]]
end

ode_f = ODEProblem(schrodinger_f, u0, tspan, pinitial);

function cost(p)
    local ode_sol = solve(ode_f, BS5(), p=Complex{Float64}.(p), abstol=1.0e-7, reltol=1.0e-7) 
    local usol = last(ode_sol)
    @views @inbounds local Ugate1 = usol[1:2, 1:2]; 
    @views @inbounds local Ugate2 = usol[4:5, 1:2];  
    @views @inbounds local Ugate3 = usol[7:8, 1:2];
    @views @inbounds local Ugate4 = usol[9:10, 1:2];
    @views @inbounds local Ugate5 = usol[12:13, 1:2];
    @views @inbounds local intPower=usol[15,1]
   @inbounds local loss = 0.003*abs(intPower)+2+(abs(1.0-((tr(Ugate1*Utarget')/2)))+abs(1.0-((tr(Ugate2*Utarget')/2)))+abs(1.0-((tr(Ugate3*Utarget')/2)))+abs(1.0-((tr(Ugate4*Utarget')/2))))+abs(1.0-((tr(Ugate5*Utarget')/2)))) 
    return loss
end

plot_callback = function (p,l) #input current params p and cost l; stores and plots history of cost function throughout optimization
    append!(cost_history,l)
    plot(log10.(cost_history), xtickfont=18, ytickfont=18, legendfont=18);
    return false
end

Optimizatoin\
The initial parameters are optimized using Radam and BFGS optimizers (more optimizations loops can be added if needed).\
The cost history is initialized and reset first then the while running DiffEqFlux it is possible to see the costfunction value while the optimizer is running.

In [ ]:
cost_history=[] 

In [ ]:
@time dnn_sol=DiffEqFlux.sciml_train(cost,pinitial, RADAM(0.002),cb=plot_callback, progress=true,maxiters=20);

In [ ]:
@time dnn_sol2=DiffEqFlux.sciml_train(cost,dnn_sol.minimizer, BFGS(initial_stepnorm=0.001),cb=plot_callback,maxiters=50);

Next we plot the resulting pulse (works for any number of neighboring vertices)

In [ ]:
soll=2vertex_Leak_Xgate;#dnn_sol2.minimizer for direct optimization result plotting
function Omegax(t)
    Ωx =0
    p1=soll
    for i in 1:int(numberF/2)
        local sina=sin(pi*i*t/T)
        Ωx=Ωx+p1[2*i-1]*sina
    end  
    return Ωx
end 

function Omegay(t)
    Ωy = 0
    p1=soll
    for i in 1:int(numberF/2)
        local sina=sin(pi*i*t/T)
        Ωy=Ωy+p1[2*i]*sina
    end  
    return Ωy
end

maxpower(t)=Ωmax;
minpower(t)=-Ωmax;
power(t)=sqrt(Omegax(t)^2+Omegay(t)^2);
plot([Omegax Omegay  power maxpower minpower] ,0,T) ;

Calculating the resulting infidelity of the optimized pulses and plotting them \
The funciton are different depending on the number of nearest neighbor vertices.

In [ ]:
# Trace infidelity function for 2 nearest neighbors
soll=2vertex_Leak_Xgate# or (dnn_sol2.minimizer) this is the weights of the pulse tested here
# See below for the weights of the gates presented in the paper 

numberF=length(soll) #in case switching between optimizations here is a good place to check the number of Sin series terms taken.

function schrodinger_ϵ(u,p,t) #The schrodinger equation again but now the extra function p is used to see how much Δ can fluctuate
    Ωx, Ωy =[0 0]
    for i in 1:int(numberF/2)
        local sina=sin(pi*i*t/T)
        Ωx=Ωx+soll[2*i-1]*sina
        Ωy=Ωy+soll[2*i]*sina
    end
    local H = Hermitian(Ωy/2*(IIY+1*IIY2)+Ωx/2*(IIX+1*IIX2)+Jmax/4*(Bigcouple))+(Δ+p)*(LII+ILI+IIL);    
    return -im*(H)*u
end

prob = ODEProblem(schrodinger_ϵ, U0, tspan)

function infidelity(ϵ) #Infidelity outputs the trace infidelity as a function of how far the real Δ is from the value assumed at the top
    sol = solve(prob, BS5(), p=ϵ, abstol=1e-9, reltol=1e-9)
    @views Ugate = last(sol)[1:27,1:27]
    return abs(1.0-((tr(UtargetAll'*Ugate*III0)/8)))
end 

In [ ]:
# Trace infidelity function for 3 nearest neighbors

In [ ]:
# Trace infidelity function for 4 nearest neighbors

In [ ]:
infidelity(0.000)

We can also plot the infidelity as a function of this offset in Δ

In [ ]:
Plots.plot([infidelity x->x^2 x->x^4], 0.001, 1, yaxis=:log10,xaxis=:log10,ylims=(1e-4,1e-0),ylabel="Infidelity",xlabel=L"\delta \Delta",xguidefontsize=18,yguidefontsize=18,xtickfont=13, ytickfont=18, legendfont=18,legend=:topleft);

The weights for the optimized pulses resulting in X gates in the presence of leakage are reported below: 

In [ ]:

2vertex_Leak_Xgate=[-0.25106804052553955,-0.6362220228548789,-0.2272883002192407,0.6014721389531674,-0.006812001385326286,-0.6007077382050238,-0.039900768594424096,0.7523786952108267,0.08619478030218519,-0.4708802204555503,0.07703590223094772,0.242888579645735,-0.06136253500636398,0.028068167109081044,0.07330559026556314,-0.27948926433980975,-0.1780912963193262,0.42598908542918795,0.0965709853447324,-0.35289665705912315,-0.09913586027064362,0.3078800049336651,-0.06970621988313433,-0.34427719132563095,0.2890836962660642,0.15337925510935602,-0.4695089851934362,0.10928502143238553,0.3114297644144096,-0.16566186650093637,-0.38192888601104913,0.1864429580014988,0.27569476206019117,-0.1421269697384641,-0.25604020959389734,0.25840304311069234,-0.00990476962371877,-0.19989796039431032,0.0983854079724941,0.09863118741169624,-0.0797050875916127,0.028125361822237724,0.004938426213263799,-0.06053835974013428,0.20810952100354163,-0.10280884576522563,0.2045077057836067,0.25482969752969,-0.04690947641639724,0.09820006622727592,0.27771497630257475,0.012641171163306927,0.19239953529440149,0.626769394104916,-0.43918045134247463,0.20851034004394664,0.22373340724302446,0.2524037251325854,-0.7743538696013426,0.7255360511574125,-0.7089894776511012,-0.6296392996449209,0.07029447947018434,-0.08545191172569686,-0.5866562776861324,-0.26721564782854584,0.16506403961137917,-0.6573950471471941,0.033211858737471615,-0.012408139530537647,-0.043760881787621345,-0.6153313436932177,0.5227616288147501,-0.1321024955912741,-0.07632654522842883,-0.18219741316224203,0.565700064376852,-0.3729788644699814,0.16537808049995648,0.1802405710252019,0.3203436495880446,-0.5841195578108733,1.0606016302581365,0.24436370583355005,0.0762820829812155,0.6116576443301412,0.3092082628696082,-0.023363364628960465,0.36165659618638496,0.697979847937143,-0.31764812113573837,0.28916295238316864,0.1686320306923256,0.14608382580093768,-0.20125290866174467,0.3856222087106447,-0.15314459564709598,-0.08719913525869948,0.10090793965476975,0.12510179822377474];
3vertex_Leak_XGate=[0.042240995021841234,0.09547778483517588,-0.05338973077223749,0.002643215791620325,-0.03164407243530625,-0.03212262152898944,0.01650032658613348,-0.052062092706450874,0.04963296471681271,0.0427442510395187,-0.065094936651914,-0.1640886129148306,0.2970622286601863,-0.033279606151716476,-0.003721541553015262,0.24941102480831037,-0.0847196064531111,-0.0355038794163105,0.007429143638921854,-0.045635317576093695,-0.06537996639458504,0.03070574686812814,0.02547224770677335,-0.22874289347129,0.28379198003251704,0.06972473121136251,-0.03624747299074921,0.19265331729188748,-0.067798146493421,0.020609673080067285,-0.01437003530708576,0.04598488884971562,-0.1717160790207015,0.008848105522147601,-0.0017189947645386246,-0.2442933420105547,0.21960773992526492,0.035896391300175995,-0.04037450606775654,0.11521082228916049,0.008870703561943435,0.013814458856078796,-0.06569233169604959,0.19828557256795618,-0.33431140007936283,-0.0630549596995505,0.02580962037464424,-0.25452668415129204,0.015826336968710816,0.004595976756941908,-0.010701963516325597,-0.1448969921931372,0.0632789405728032,-0.028180805593509644,0.03810256118241418,-0.09414823876257394,0.07649201385365247,0.026197434376108913,-0.0014363120357830694,-0.06678079996283214,0.08985036723092184,0.008005730922472863,0.009779197622044149,-0.02621903051311326,0.06601964726529448,0.023419731012336107,-0.003377779565671683,-0.02727616598308091,0.07014968192631191,0.02077336307538232,-0.007859265944021906,-0.01154774214222906,0.0616388245811985,0.020372188337932674,-0.017392943597359614,-0.003307826081337598,0.060075924000773544,0.00946334859450853,-0.013779861975682588,0.017303070152404953,0.03194968289595227,-0.0004289268364605548,0.007906663352485547,0.0046114261899777515,0.031564560199314716,0.02555725643898603,-0.009466459066241224,0.0049923477201592945,0.035430369786864015,0.02486919921379354,-0.02150478405703305,0.025063972476125258,0.014441379132544261,0.0076540781406900455,-0.0042397646985846875,0.02969548076132915,-0.006517351296143448,0.018514702642748123,0.008131731122630876,0.02318673018010035,-0.014156018172585874,0.066325749490361,-0.0625406980274718,0.027675358940263052,-0.02233815732651308,0.004476719926846261,-0.03031961060534828,0.06425216871382064,-0.11749112548906539,0.015437408516943647,-0.03555743009719376,-0.04061583684306423,-0.03175220500545899,0.01171622240668286,-0.025009003208173317,0.08869979390478348,-0.28466846639549886,0.03830080887669875,-0.08352617899767356,-0.24736748846928186,-0.0023523763798395684,-0.054865328672965404,0.03089009164498998,-0.07220132388757401,-0.18352547153645135,0.06732131573495763,-0.09344886464286929,-0.39140497588053225,0.23833212018365185,-0.07336053919423383,0.059885962047812755,-0.028713194715760733,0.026364142824139285,0.02771871211555941,0.12666275981374006,-0.16012135702772085,0.1932603824129575,0.32541175387225146,-0.2821840408548982,0.06633791894216862,-0.017708882211501836,-0.014729252172755192,-0.056381293285696195,-0.03491963748244958,-0.07873900149702373,0.30770260634391555,-0.6383792127609079,-0.24163504853910905,0.17630092375560844,-0.41405284668103504,-0.0971616689046783,-0.06877420430104086,0.1489319355525357,-0.23733739770534515,-0.07844619641389317,-0.006784156444949463,0.10074755565089935,-0.2933697381670481,0.095294178241092,0.0024062802317780995,0.06415371564228393,-0.13330104125243253,0.06679996560600117,-0.025840574097617988,0.08042260660543837,-0.08634900682629693,0.03556938103905237,-0.01701871185063898,0.08527653700120985,-0.0980821195709648,0.05940106477632077,0.002268062795359206,0.06477282387996537,-0.0797064835843045,0.08042176694817953,-0.010726713908825895,0.07431125790532501,-0.043386974571717904,0.08192236391313769,-0.003726299036331065,0.07679849229863077,-0.009070968569890824,0.049280818682174495,0.006391012090934776,0.07228036658663144,-0.005032016769499809,0.034384769482198305,0.011261832355865701,0.06945629739448689,-0.0021697870638738423,0.02429540069275641,0.014947471354619293,0.07020860703352201,0.0006398138359334815,0.01596440527732811,0.02273600659310401,0.07292034650813302,0.0061019460321652536,-0.0012024348399861677,0.047268890463526524];
4vertex_Leak_XGate=[4.917070396215213,4.92055372101269,0.2915143666249287,0.3088079624153355,1.4119099615266517,1.3686071998771545,1.6026295505478647,1.5858553003621574,1.45403670266228,1.4407029666458722,0.0642123189461131,0.05940371698026733,-0.08225340532033978,-0.04553489801634831,-0.011544090143431447,-0.06799798332804846,0.12461493744848959,0.07822547239983013,0.17576362128719344,0.09108881840559811,0.05524925537352018,0.0728282973119962,0.10881127582388654,-0.00989465948950713,0.12951189430658946,0.153498377678741,-0.0286078127465335,-0.06687237082413344,0.1192393360058435,0.15173757586397527,-0.04813976794892009,-0.03492178226054097,0.1543404912141444,0.01271308057427451,-0.014323343840678382,-0.09009798605404518,0.19248704327804186,-0.01479796777879017,-0.05215132745993327,0.06739704151455807,0.13294087582643427,0.023124876126655067,-0.07617357356509957,0.09977764762527562,0.1787056895605114,-0.09899559268473468,-0.03570288961129107,0.05019343314078303,0.3000484784636601,-0.15872243271995873,0.031058046434097656,0.20447856177232318,0.13023557843477232,-0.07415576139161792,-0.042967880292637234,0.24090591367483738,0.062349784014629754,-0.04735507883371376,0.01568501336764091,0.07792570582354502,0.10505470939745308,-0.1824366869581105,0.14469035628819074,0.08073166763586331,0.19772772556208357,-0.10735736040448793,0.05720371994824884,0.29593096040361383,-0.08617022143050203,0.04793205995578356,0.2240094740287312,0.13780804455705642,-0.3067212602478662,0.26933803825573127,0.23274922467663534,-0.3028135176708661,0.07213299079940297,0.21822905815208032,-0.04855567859667247,-0.1114528899834512,0.4270131306187203,0.059096385328028994,-0.18526496853149793,0.3577811903385549,-0.0376295624545246,0.06084940175125393,-0.1299952746630809,0.14952534371329784,-0.012842776777620041,0.03261326153571618,0.06770904694337763,-0.04103474908870837,0.26209330675506853,0.18306247632926284,-0.2633161281323076,0.2762021589304553,0.16382809258696793,-0.1462287036503816,-0.1476095125567749,0.22043792790096123,-0.011803156402144606,-0.30287017537938454,0.23686315903604963,0.17622084934230497,0.0022700883602609625,0.05883605108501912,0.028648908435413567,0.2684852161979816,-0.19953433080084318,0.16293799437262932,-0.08528043165491972,0.1326222791306475,-0.0915802913920881,-0.07881487679526948,-0.10197582527259479,0.1538198626919539,-0.19541018665131912,-0.24657040766798216,0.3548640718059427,-0.21442314496849305,0.09009322944849055,0.14097364788640734,-0.09534242536004607,0.13118412768075266,-0.07735311238739326,-0.10999723151246875,0.13743815330466339,-0.08701346817663844,-0.04268487880565773,0.08453649061121736,-0.04413581944260188,-0.04302782975687583,0.13788121720940508,-0.1464083689634178,0.16644775541943738,0.11234947974660169,-0.10472181469868132,0.09004514543874,0.01240346853567393,-0.11647157915008097,0.1195299609865527,-0.11521689711625759,0.17582933679442111,0.00667161082377548,0.052734027700788194,0.001236014060176936,0.230640256005251,-0.05562336982548694,0.06358866979530933,0.19864196881221052,0.034776235995871624,-0.10132031875489336,0.25456103424748544,0.15809128940157033,-0.23162253393048185,0.1785951355382927,-0.02129694106089761,-0.2562355936131065,0.31113795333713384,-0.10084909104891848,0.21340419720838721,0.23074453840052384,-0.12354033420828638,0.16670942564778476,0.09673307727492357,-0.11555629527669144,0.17306918419004963,0.08996006591612853,0.04274064519439934,0.09345333102661936,0.08550420380089115,-0.025180597533629452,0.26121072113729576,0.026328622213231883,0.13321531648287827,0.20309907135963565,0.052219802188916316,0.19722370950997267,-0.12746348836544363,0.07858605722900977,0.11437800888117092,-0.114287606022787,0.06966518976625968,0.1361904835130773,0.09259017845886608,-0.06770583690308006,0.15146223168201267,0.11896453942143889,0.13348546862099014,0.04799674337115564,0.06441180434605362,0.24677114122990199,-0.02713358650016206,-0.0044877657608064555,-0.06700733118324408,0.10008361315065278,-0.007697132563432245,-0.43764927431672457,0.5637218157973388,-0.015389898918013062];